In [78]:
import enum
from collections import Counter, defaultdict
from ntpath import join
import numpy as np
import pandas as pd
from sqlalchemy import column
import xlsxwriter
import os
import seaborn as sns
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('omw-1.4')
from pathlib import Path
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder
from matplotlib.gridspec import GridSpec
import contractions 
from pathlib import Path
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.io import imread, imshow
from skimage import io
import json
from skimage import img_as_float
from skimage.transform import rescale, resize
from itertools import product
from PIL import Image
from clean_tabular import CleanData, CleanImages
import torch


[nltk_data] Downloading package omw-1.4 to /Users/aj8/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [75]:
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', 20)
class ProductDescpMannual(CleanData):
    def __init__(self):
        super().__init__(tab_names=['Products'])
        self.label_len = len(self.major_map_decoder)
        self.product_frame = self.table_dict['Products'].copy()

    def clean_prod(self, col='product_description'):
        stop_words = set(stopwords.words('english'))
        word_lemmitizer = WordNetLemmatizer()
        punct_re = re.compile(r'[^A-Za-z \n-]') #[^A-Za-z0-9 .]
        self.product_frame[col] = self.product_frame[col].copy().str.replace(punct_re, '')
        all_text_ls = []
        main_ls = []
        for i in self.product_frame[col]:
            dum_ls = i.split()
            dum_ls = [i for i in dum_ls if i not in  stop_words]
            dum_ls = [i.replace('\n', ' ') for i in dum_ls]
            dum_ls = [i.strip('\-') for i in dum_ls]
            dum_ls = [word_lemmitizer.lemmatize(i) for i in dum_ls]
            dum_ls = [contractions.fix(j) for j in dum_ls]
            dum_ls = [i for i in dum_ls if i.isalpha] # and len(i)>2
            main_ls.append(' '.join(dum_ls))
            # print(' '.join(dum_ls))
            all_text_ls.extend(dum_ls)
        all_text_ls = [i.lower() for i in all_text_ls]

        # wrd_counter = Counter(all_text_ls)
        # wrd_freq_dict = {wrd: count/len(wrd_counter) for wrd, count in wrd_counter.items()}
        # wrd_drop_proba = {wrd: 1- np.sqrt(threshold/wrd_freq_dict[wrd]) for wrd in wrd_counter.keys()}
        # main_ls = []
        # for i in self.product_frame[col]:
        #     dum_ls = i.split()


        self.product_frame[col] = main_ls 
        self.product_frame[col] = self.product_frame[col].apply(lambda i: i.lower())
        print(len(all_text_ls))
        return all_text_ls, len(all_text_ls)
    
    def word_freq(self, col='product_description', num_items=100):
        all_words = self.clean_prod(col=col)[0]
        count_ls = Counter(all_words)
        if num_items is not None:
            count_dict = {i[0]: i[1] for i in count_ls.most_common(num_items)}
        else:
            count_dict = {i[0]: i[1] for i in count_ls.most_common()}
        return count_dict

    def get_word_set(self, col='product_description'):
        if col=='product_name':
            self.clean_prod_name(col=col)
        self.clean_prod(col=col)
        self.full_word_ls = []
        for i in self.product_frame[col]:
            self.full_word_ls.extend(i.split())
        self.full_word_set = list(set(self.full_word_ls))
        return self.full_word_ls, self.full_word_set ,len(self.full_word_set)+1

    def vocab_encoder(self, col='product_description', limit_it=False, vocab_limit=30000):
        word_ls, word_set, full_vocab_size = self.get_word_set(col)
        if limit_it==False:
            vocab_size = full_vocab_size
            self.word_encoder = defaultdict(lambda: vocab_size-1)
            current_word_encoder = {key[0]: integer for integer, key in enumerate(Counter(word_ls).most_common()[:vocab_size-1])}
            self.word_encoder['<UNKNOWN>'] = vocab_size-1
            self.word_encoder.update(current_word_encoder)
            self.word_decoder = {val: key for key, val in self.word_encoder.items()}
        else:
            vocab_size=vocab_limit
            self.word_encoder = defaultdict(lambda: vocab_size)
            current_word_encoder = {key[0]: integer for integer, key in enumerate(Counter(word_ls).most_common()[:vocab_size])}
            self.word_encoder['<UNKNOWN>'] = vocab_size
            self.word_encoder.update(current_word_encoder)
            self.word_decoder = {val: key for key, val in self.word_encoder.items()}
        
        # self.word_encoder = {val: key for key, val in self.word_decoder.items()}
        return self.word_encoder, self.word_decoder, self.product_frame

    
    def dataloader_preprocess(self, limit_it=True, vocab_lim=10000, context_size=5):
        product_encoder, product_decoder, DF = self.vocab_encoder(limit_it=limit_it, vocab_limit=vocab_lim)
        df = DF.copy()
        init_ls = []
        for i in range(len(df)):
            prod_descript = df['product_description'].iloc[i].split()
            init_ls.append([
                [
                    list(np.array([[prod_descript[i - j], prod_descript[i+j]] for j in range(1, context_size+1)]).flatten()),
                    prod_descript[i]
                ]
                for i in range(context_size, len(prod_descript)-context_size)
            ])
        df['original_context_target'] = init_ls
        df = df.explode('original_context_target').reset_index()
        for idx, val in enumerate(df['original_context_target']):
            if type(val) is not list:
                df.drop(idx, axis=0, inplace=True)
        
        def stub_coder(ls, code_type='encoded'):
            code_dict = {'encoded': product_encoder, 'decoded': product_decoder}
            new_ls = []
            for i in ls.copy():
                new_ls.append(code_dict[code_type][i])
            return new_ls

        df = pd.concat([df, pd.DataFrame(df['original_context_target'].tolist())], axis=1)
        df.rename(columns={0: 'context', 1: 'target'}, inplace=True)
        df.dropna(inplace=True)
        df['context_encoded'] = df['context'].apply(lambda i: stub_coder(ls=i, code_type='encoded'))
        df['context_decoded'] = df['context_encoded'].apply(lambda i: stub_coder(ls=i,code_type='decoded'))
        df['target_encoded'] = df['target'].apply(lambda i: product_encoder[i])
        df['target_decoded'] = df['target_encoded'].apply(lambda i: product_decoder[i])
        print(df)
        print(df.columns)
        self.product_frame = df.copy()
        return self.product_frame, product_encoder, product_decoder





In [ ]:
def generate_batches(batch_size=32, context_size = 4, threshold=1e-5):
    product_description_class = ProductDescpMannual()
    DF, product_encoder, product_decoder, total_wrd_ls = product_description_class.dataloader_preprocess(limit_it=True, vocab_limit=10000, context_size=4)
    df = DF.copy()
    num_batches = len(df)//batch_size
    corpus_length = len(total_wrd_ls)
    df = df.iloc[:num_batches*batch_size]
    wrd_counter = Counter(total_wrd_ls)
    wrd_freq_dict = {wrd: count/len(total_wrd_ls) for wrd, count in wrd_counter.items()}
    wrd_drop_proba = {wrd: 1- np.sqrt(threshold/wrd_freq_dict[wrd]) for wrd in wrd_counter.keys()}
    
    for idx in range(0, len(df), batch_size):
        pa
    

In [ ]:
prod = ProductDescpMannual()
prod.dataloader_preprocess()

In [ ]:
def train_model():
    product_description_class = ProductDescpMannual()
    DF, product_encoder, product_decoder, total_wrd_ls = product_description_class.dataloader_preprocess(limit_it=True, vocab_limit=10000, context_size=4)
    df = DF.copy()
    corpus_length = len(total_wrd_ls)


## BERT TRANSFORMER

In [61]:
from transformers import BertTokenizer
from transformers import BertModel

In [77]:
class TextDatasetBert(torch.utls.data.Dataset):
    def __init__(self, max_length, min_count=2):
        prod = ProductDescpMannual()
        self.product_df, self.mannual_word_encoder, self.mannual_word_decoder = prod.dataloader_preprocess()
        current_vocab = prod.full_word_set
        currnet_corpus = prod.full_word_ls
        current_vocab_size = len(current_vocab) + 1

        self.label_encoder = prod.major_map_encoder
        self.label_decoder = prod.major_map_decoder
        self.labels = self.product_df['major_category_encoded'].to_list()

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self.max_length = max_length

        product_description_counter = Counter(current_corpus)
        main_ls = []
        for i in self.product_df['product_description'].copy():
            temp_ls = i.split()
            temp_ls = [i for i in temp_ls if product_description_counter[i]>min_count]
            main_ls.append(' '.join(temp_ls))
        self.product_df['product_description'] = main_ls
        self.product_descriptions = self.product_df['product_description'].to_list()
        

    def __getitem__(self, idx):
        label = torch.as_tensor(self.labels[idx])
        descript = self.product_descriptions[idx]
        bert_encoded = self.tokenizer.batch_encode_plus([descript], max_lenght=self.max_length, padding='max_length', truncation=True)
        bert_encoded = {key: torch.LongTensor(value) for key, value in bert_encoded.items()}
        with torch.no_grad():
            new_description = self.model(**bert_encoded).last_hidden_state.swapaxes(1,2)
        
        new_description = new_description.squeeze(0)
        return new_description, label



        

NameError: name 'torch' is not defined

[('a', 3), ('b', 2), ('c', 1)]